## 4.2 Up in the clouds — Solveur d'allocation jobs/serveurs

### Règles

- **1 job par serveur, 1 serveur par job** (matching bipartite parfait)
- **Serveur valide** si toutes les conditions sont respectées :
  - `cpu_s >= cpu_j`
  - `ram_s >= ram_j`
  - `disk_s >= disk_j`
- **Coût d'allocation** = `runtime_j × cpu_j × server_cost`

### Objectif

Trouver le **coût total minimal** pour allouer tous les jobs aux serveurs.

### Approche algorithmique

Utiliser **Min-Cost Max-Flow** (OR-Tools) sur un graphe bipartite :

1. **Construction du graphe** :
   - Nœuds gauche : n jobs (supply = +1 chacun)
   - Nœuds droits : n serveurs (supply = -1 chacun)
   - Arcs job → serveur si le serveur est valide (capacité = 1)
   - Coût de l'arc = `cpu_j × runtime_j × server_cost`

2. **Résolution** :
   - Appliquer l'algorithme Min-Cost Max-Flow
   - Scaling des coûts (×10 000) pour garder la précision avec entiers
   - Récupérer le coût optimal et rescaler

In [ ]:
import re
from pathlib import Path
from ortools.graph.python import min_cost_flow

NUM_RE = re.compile(r"\(([^)]+)\)")

def parse_line_tuple(line: str):
    m = NUM_RE.search(line)
    if not m:
        raise ValueError(f"Ligne invalide: {line!r}")
    parts = [p.strip() for p in m.group(1).split(",")]
    if len(parts) != 4:
        raise ValueError(f"Tuple invalide: {line!r}")
    return tuple(float(x) for x in parts)

def load_jobs(path: str):
    jobs = []
    for ln in Path(path).read_text(encoding="utf-8").strip().splitlines():
        cpu, ram, disk, runtime = parse_line_tuple(ln)
        jobs.append((int(cpu), int(ram), float(disk), float(runtime)))
    return jobs

def load_servers(path: str):
    servers = []
    for ln in Path(path).read_text(encoding="utf-8").strip().splitlines():
        cpu, ram, disk, cost = parse_line_tuple(ln)
        servers.append((int(cpu), int(ram), float(disk), float(cost)))
    return servers

def feasible(job, srv):
    cpu_j, ram_j, disk_j, _ = job
    cpu_s, ram_s, disk_s, _ = srv
    return (cpu_s >= cpu_j) and (ram_s >= ram_j) and (disk_s + 1e-12 >= disk_j)

def main():
    jobs = load_jobs("res/jobs.txt")
    servers = load_servers("res/server.txt")
    n = len(jobs)
    assert n == len(servers)

    SCALE = 10_000

    # Nœuds:
    # 0..n-1   = jobs
    # n..2n-1  = servers
    # on crée un flot de 1 par job, demande = +1 ; chaque serveur demande = -1
    mcf = min_cost_flow.SimpleMinCostFlow()

    # supplies
    supplies = [0] * (2 * n)
    for j in range(n):
        supplies[j] = 1
    for s in range(n):
        supplies[n + s] = -1

    # Arcs jobs -> servers si feasible
    arc_count = 0
    for j in range(n):
        cpu_j, ram_j, disk_j, runtime_j = jobs[j]
        a = cpu_j * runtime_j  # facteur du job
        for s in range(n):
            srv = servers[s]
            if feasible(jobs[j], srv):
                cost_s = srv[3]
                cost = int(round(a * cost_s * SCALE))
                mcf.add_arc_with_capacity_and_unit_cost(j, n + s, 1, cost)
                arc_count += 1

    # Ajout supplies
    for node, sup in enumerate(supplies):
        mcf.set_node_supply(node, sup)

    status = mcf.solve()
    if status != mcf.OPTIMAL:
        raise RuntimeError(f"Pas de solution optimale. Statut OR-Tools={status}")

    total_scaled = mcf.optimal_cost()  # somme des coûts entiers (scaled)
    total = total_scaled / SCALE
    print(f"{total:.2f}")

if __name__ == "__main__":
    main()


3575.64
